# Multi-class classification with Softmax regression

- https://www.youtube.com/watch?v=MFAnsx1y9ZI&index=13&list=PLlMkM4tgfjnLSOjrEJN31gZATbcj_MpUm

- multinomial classification = 3 개의 독립된 클래시파이어를 만들면 즉 각 클래서를 선택하는 클래시파이어를 만드는 것인데, 그냥 W 행렬을 1x3 짜리 3개를 만들지 말고 3x3 짜리 한 개를 만들자. 그러면, $Y= X * W$ 에서 얻어지는 $Y$는 3차원 벡터가 된다.
- 그러면 $Y$ 값을 확률값으로 변경하고자하면 어떻게 해야하나. softmax 함수를 사용하면 된다.
- $Softmax(y_i) = \frac{\exp{y_i}}{\sum \exp{y_k}}$
- https://www.tensorflow.org/api_docs/python/tf/nn/softmax
- ``log_softmax`` 가 정의되어 있다. https://www.tensorflow.org/versions/r0.10/api_docs/python/nn/classification
- https://www.tensorflow.org/get_started/mnist/beginners
- https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/tutorials/mnist/mnist_softmax.py : may be numerically unstable so, ``tf.nn.softmax_cross_entropy_with_logits()`` 를 대신 사용한다.

## 학습을 위한 코스트 함수는?
- cross-entropy 를 사용하자
- $D (S, L) = -\sum_i L_i \log(S_i)$, where $S$ is the prediction output vector from softmax and $L$ is the label vector (e.g. [0, 1, 0]).

### Ref.
- http://stackoverflow.com/questions/40675182/tensorflow-log-softmax-tf-nn-logtf-nn-softmaxpredict-tf-nn-softmax-cross-ent

In [1]:
import numpy as np
import tensorflow as tf

In [2]:
xy = np.loadtxt('train-softmax.txt', unpack=True, dtype='float32')
xdata = np.transpose(xy[0:3])
ydata = np.transpose(xy[3:])
print ('xdata: ', xdata)
print ('ydata: ', ydata)
print ('notice that each row corresponds to one data input')

xdata:  [[ 1.  2.  1.]
 [ 1.  3.  2.]
 [ 1.  3.  4.]
 [ 1.  5.  5.]
 [ 1.  7.  5.]
 [ 1.  2.  5.]
 [ 1.  6.  6.]
 [ 1.  7.  7.]]
ydata:  [[ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]]
notice that each row corresponds to one data input


In [3]:
# TF Graph Input
X = tf.placeholder ("float", [None,3])
Y = tf.placeholder ("float", [None,3])

input_dim = 3
output_dim = 3
W = tf.Variable(tf.random_uniform([input_dim, output_dim], -1,1))
hypothesis = tf.nn.softmax (tf.matmul(X, W))

#cross entropy
cost = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(hypothesis), axis=1))

```
cost = tf.reduce_mean( 
    tf.nn.softmax_cross_entropy_with_logits (labels=Y, logits=hypothesis))
```
learning_rate = 0.1
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

with tf.Session() as ss:
    ss.run(tf.global_variables_initializer())
    feed_dict = {X:xdata[:2], Y:ydata[:2]}
    print ('W:', ss.run(W))
    print ('X', ss.run(X, feed_dict = {X:xdata}))
    print ('Y', ss.run(Y, feed_dict = {Y:ydata}))
    print ('softmax(X*W)', ss.run(tf.nn.softmax(tf.matmul(X,W)), 
                                  feed_dict={X:xdata[:2]}))
    print ('hypothesis=', ss.run(hypothesis, feed_dict))
    print ('tf.log(hypothesis)=',
          ss.run(
          tf.log(hypothesis), feed_dict))
    print ('Y*tf.log(hypothesis)=',
          ss.run(Y*tf.log(hypothesis), feed_dict))
    print ('tf.reduce_sum(Y*tf.log(hypothesis), axis=1)=',
          ss.run(tf.reduce_sum(Y*tf.log(hypothesis), axis=1), feed_dict))
    print ('cost = tf.reduce_mean(-tf.reduce_sum) = ', 
           ss.run(cost, feed_dict))

    feed_dict = {X:xdata, Y:ydata}
    
    for step in range(20001):
        ss.run(optimizer, feed_dict)
        if step%2000==0:
            print ('{}/2001 cost={} '.format(
                step,
                ss.run(cost, feed_dict))
            )
            
    # see the learning result
    print ('--- learning result ---')
    re = ss.run(hypothesis, feed_dict)
    rIndx = [ss.run(tf.arg_max(re,1))]
    print ('hypothesis=', re)
    print ('predicted Indx=', rIndx)
    
    testX = [ [1,11,7], [1,3,4], [1,1,0]]
    print ('test output: ', ss.run(tf.arg_max(hypothesis,1), 
                                   feed_dict={X:testX}))

SyntaxError: invalid syntax (<ipython-input-3-806eecf638f5>, line 13)

## 결론

대략 돌아가는 상황을 파악하는 용도로만 사용하자. 데이터 8개로 피팅이 정말 잘 될 거라고 생각하지는 말아야하지않을까?